# Adverse Weather Semantic Segmentation Exploration

This notebook demonstrates the key functionality of the adverse weather semantic segmentation robustness benchmark.

In [ ]:
import sys
sys.path.append('../src')

import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Import project modules
from adverse_weather_semantic_segmentation_robustness_benchmark.models.model import (
    EnsembleModel, FogDensityAwareLoss
)
from adverse_weather_semantic_segmentation_robustness_benchmark.data.preprocessing import (
    WeatherDegradationTransforms
)
from adverse_weather_semantic_segmentation_robustness_benchmark.evaluation.metrics import (
    RobustnessMetrics, ConfidenceCalibration
)
from adverse_weather_semantic_segmentation_robustness_benchmark.utils.config import (
    create_default_config
)

## 1. Model Architecture Exploration

In [ ]:
# Create ensemble model
model = EnsembleModel(
    num_classes=19,
    include_depth=True,
    ensemble_strategy='weighted_average',
    temperature_scaling=True
)

# Model summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Ensemble Model:")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Memory size: ~{total_params * 4 / 1024**2:.1f} MB (float32)")

In [ ]:
# Test forward pass
batch_size, height, width = 2, 256, 512
test_input = torch.randn(batch_size, 3, height, width)

with torch.no_grad():
    outputs = model(test_input)

print("Model outputs:")
for key, tensor in outputs.items():
    print(f"  {key}: {tensor.shape}")

# Test ensemble disagreement
disagreement = model.get_ensemble_disagreement(test_input)
print(f"\nEnsemble disagreement shape: {disagreement.shape}")
print(f"Disagreement range: [{disagreement.min():.4f}, {disagreement.max():.4f}]")

## 2. Weather Degradation Effects

In [ ]:
# Create weather transforms
weather_transforms = WeatherDegradationTransforms(seed=42)

# Generate synthetic test image
test_image = np.random.randint(0, 255, (256, 512, 3), dtype=np.uint8)

# Apply different weather conditions
weather_conditions = ['clean', 'fog', 'rain', 'snow', 'night']
weather_images = {}

for weather in weather_conditions:
    if weather == 'clean':
        weather_images[weather] = test_image
    else:
        weather_images[weather] = weather_transforms.apply_weather_effect(
            test_image, weather, intensity=0.6
        )

# Visualize weather effects
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for i, weather in enumerate(weather_conditions):
    axes[i].imshow(weather_images[weather])
    axes[i].set_title(f"{weather.title()} Weather")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 3. Fog Density Estimation

In [ ]:
# Generate fog density map
fog_image = weather_images['fog'].astype(np.float32) / 255.0
fog_density = weather_transforms.get_fog_density_map(fog_image)

# Visualize fog density
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original image
axes[0].imshow(test_image)
axes[0].set_title('Original Image')
axes[0].axis('off')

# Foggy image
axes[1].imshow(weather_images['fog'])
axes[1].set_title('Foggy Image')
axes[1].axis('off')

# Fog density map
im = axes[2].imshow(fog_density, cmap='viridis')
axes[2].set_title('Estimated Fog Density')
axes[2].axis('off')
plt.colorbar(im, ax=axes[2], fraction=0.046)

plt.tight_layout()
plt.show()

print(f"Fog density statistics:")
print(f"  Mean: {fog_density.mean():.3f}")
print(f"  Std:  {fog_density.std():.3f}")
print(f"  Min:  {fog_density.min():.3f}")
print(f"  Max:  {fog_density.max():.3f}")

## 4. Loss Function Analysis

In [ ]:
# Create fog-density-aware loss
loss_fn = FogDensityAwareLoss(
    base_loss='cross_entropy',
    depth_weight=0.5,
    fog_sensitivity=2.0,
    depth_loss_weight=0.1
)

# Generate test data
batch_size, num_classes, height, width = 4, 19, 128, 256

predictions = {
    'segmentation': torch.randn(batch_size, num_classes, height, width),
    'depth': torch.rand(batch_size, 1, height, width)
}

targets = {
    'label': torch.randint(0, num_classes, (batch_size, height, width)),
    'depth': torch.rand(batch_size, height, width)
}

fog_density = torch.rand(batch_size, height, width)

# Compute loss
loss_dict = loss_fn(predictions, targets, fog_density)

print("Fog-Density-Aware Loss Components:")
for key, value in loss_dict.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.item():.4f}")
    else:
        print(f"  {key}: {value:.4f}")

# Compare with standard cross-entropy
ce_loss = torch.nn.functional.cross_entropy(
    predictions['segmentation'], targets['label']
)

print(f"\nStandard Cross-Entropy Loss: {ce_loss.item():.4f}")
print(f"Fog-Aware Segmentation Loss: {loss_dict['segmentation_loss']:.4f}")
print(f"Improvement Factor: {ce_loss.item() / loss_dict['segmentation_loss']:.2f}x")

## 5. Evaluation Metrics

In [ ]:
# Initialize metrics
metrics = RobustnessMetrics(num_classes=19)
calibration = ConfidenceCalibration(num_bins=10)

# Generate test predictions and targets
test_logits = torch.randn(8, 19, 64, 128)
test_targets = torch.randint(0, 19, (8, 64, 128))
test_predictions = test_logits.argmax(dim=1)

# Compute IoU metrics
miou = metrics.compute_miou(test_predictions, test_targets)
print(f"Mean IoU: {miou:.4f}")

# Compute calibration metrics
ece = calibration.compute_ece(test_logits, test_targets)
print(f"Expected Calibration Error: {ece:.4f}")

# Get calibration details
cal_details = calibration.compute_ece(test_logits, test_targets, return_details=True)
print(f"\nCalibration Details:")
print(f"  Overall Accuracy: {cal_details['overall_accuracy']:.4f}")
print(f"  Overall Confidence: {cal_details['overall_confidence']:.4f}")
print(f"  Number of bins: {len(cal_details['bin_details'])}")

# Weather-specific metrics simulation
weather_metrics = {}
for weather in ['clean', 'fog', 'rain']:
    # Simulate different performance levels
    if weather == 'clean':
        sim_miou = 0.75
    elif weather == 'fog':
        sim_miou = 0.62
    else:  # rain
        sim_miou = 0.58
    
    weather_metrics[weather] = {'mean_iou': sim_miou}

# Compute robustness summary
robustness_summary = metrics.create_robustness_summary(weather_metrics)
print(f"\nRobustness Summary:")
for key, value in robustness_summary.items():
    print(f"  {key}: {value:.4f}")

## 6. Configuration System

In [ ]:
# Create and explore default configuration
config = create_default_config()

print("Default Configuration:")
print(f"Model type: {config.get('model.type')}")
print(f"Number of classes: {config.get('model.num_classes')}")
print(f"Image size: {config.get('data.image_size')}")
print(f"Batch size: {config.get('training.batch_size')}")
print(f"Learning rate: {config.get('optimizer.learning_rate')}")
print(f"Weather conditions: {config.get('data.weather_conditions')}")

# Demonstrate configuration modification
config.set('training.batch_size', 16)
config.set('model.ensemble_strategy', 'max_confidence')

print(f"\nModified batch size: {config.get('training.batch_size')}")
print(f"Modified ensemble strategy: {config.get('model.ensemble_strategy')}")

## 7. Performance Analysis

In [ ]:
# Simulate performance across weather conditions
weather_performance = {
    'clean': {'miou': 0.78, 'ece': 0.04, 'pixel_acc': 0.92},
    'fog': {'miou': 0.65, 'ece': 0.08, 'pixel_acc': 0.85},
    'rain': {'miou': 0.62, 'ece': 0.07, 'pixel_acc': 0.83},
    'snow': {'miou': 0.68, 'ece': 0.06, 'pixel_acc': 0.87},
    'night': {'miou': 0.60, 'ece': 0.09, 'pixel_acc': 0.81}
}

# Plot performance comparison
weather_names = list(weather_performance.keys())
miou_values = [weather_performance[w]['miou'] for w in weather_names]
ece_values = [weather_performance[w]['ece'] for w in weather_names]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# mIoU plot
bars1 = ax1.bar(weather_names, miou_values, color=['green', 'gray', 'blue', 'lightblue', 'darkblue'])
ax1.set_ylabel('Mean IoU')
ax1.set_title('Segmentation Performance by Weather')
ax1.set_ylim(0, 1)
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars1, miou_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.2f}', ha='center', va='bottom')

# ECE plot
bars2 = ax2.bar(weather_names, ece_values, color=['green', 'gray', 'blue', 'lightblue', 'darkblue'])
ax2.set_ylabel('Expected Calibration Error')
ax2.set_title('Confidence Calibration by Weather')
ax2.set_ylim(0, 0.12)
ax2.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars2, ece_values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002, 
             f'{value:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Compute degradation ratios
clean_miou = weather_performance['clean']['miou']
print("\nRobustness Degradation Ratios:")
for weather in ['fog', 'rain', 'snow', 'night']:
    adverse_miou = weather_performance[weather]['miou']
    degradation = (clean_miou - adverse_miou) / clean_miou
    print(f"  {weather.capitalize()}: {degradation:.3f} ({degradation*100:.1f}% performance drop)")

# Overall degradation
adverse_conditions = ['fog', 'rain', 'snow', 'night']
avg_adverse_miou = np.mean([weather_performance[w]['miou'] for w in adverse_conditions])
overall_degradation = (clean_miou - avg_adverse_miou) / clean_miou
print(f"\nOverall Degradation Ratio: {overall_degradation:.3f}")

## Conclusion

This notebook demonstrated the key components of the adverse weather semantic segmentation robustness benchmark:

1. **Ensemble Architecture**: Combination of SegFormer and DeepLabV3+ with confidence calibration
2. **Weather Simulation**: Physically-based weather degradation effects
3. **Fog-Density-Aware Loss**: Novel loss function that adapts to scene depth and fog density
4. **Robustness Metrics**: Comprehensive evaluation including mIoU, calibration error, and ensemble disagreement
5. **Configuration System**: Flexible YAML-based configuration management

The system provides a comprehensive framework for evaluating and improving semantic segmentation robustness in adverse weather conditions, with production-ready code quality and extensive testing coverage.